In [ ]:
คือมานั่งเขียนไฟล์นี้เพราะว่า
ไฟล์ก่อน (feed_back_to_flask) ได้ recommended_movies มาละ
หลังจากไล่แกะทีละบรรทัดมาอย่างเมามันใน combine_dannyibo_sidthartha
ก็โอเค เสร็จงานละ เอาทุกอย่างมาเรียงกัน ถ้าเคาะ recommended_movies ออกมาได้ถือว่าจบ
ก็ได้จริงๆ แต่ไม่ได้เหมือนเดิม เลยชักงงว่ามันต้องเหมือนเดิมหรือเปล่า

แล้วตอนไล่ก็อปปี้เฉพาะบรรทัดที่ต้องใช้มาก็แอบงง ไม่รู้ตกลงเราติงต๊องหรือเปล่าเวลาไม่เห็นผลลัพธ์
แบบ ทำไมมันมา encode encoded2encode อะไรของแม่ง
ดูไม่ได้ต้องทำอะไรขนาดนั้นเลยนี่นา

คือจริงๆเพราะมีไอ้ไฟล์ all_movies
ซึ่งก็คือ movies_not_watched เนี่ยมาร่วมด้วย ก็เลยต้องพันกันนิดหน่อย
เป็นไฟล์ที่ให้เลือกตั้งแต่แรกซึ่งก็ make sense

ส่วนเรื่องเลข user embedding จริงๆมันก็เป็นหัวใจของการเทรนเลย
เพราะมันคือ x เทรนเพื่อจะไป predict ว่าอี user คนนี้มันชอบแบบไหน
มันก็ต้องมี user ขาเข้า input ไปด้วยจะได้เอามาเทียบกันถูก
ซึ่งก็คือ x นั่นแหละ
มี bias ให้ด้วย

In [ ]:
อะ คือตรงนี้ ลิสท์ที่กลับมาจาก user input มันให้เลือกด้วยลิสท์ all_movies เพราะจะให้ไปเลือกจาก all_ratings ก็คงประหลาด
ถึงแม้ว่า movieId ใน all_ratings ก็มีเหมือนกัน แต่มันไม่ต่อกัน คือเลขอินเด็กซ์มันไม่สะท้อนถึง unique list
ที่มีจำนวนหนังทั้งหมด 9742 เรื่อง

มันเป็นการสร้างความสัมพันธ์ จะนึกเป็นก้อนคิวบ์สามมิติสองก้อน แล้วมีเส้นๆเชื่อมกันเป็นเน็ตเวิร์คก็ได้
แล้วแต่ละจุด ก็พล็อตด้วยเลขเว็คเตอร์ ซึ่งก็คือ embedding นั่นเอง
แล้วเส้นความสัมพันธ์ก็คือ weight
ว่าได้เท่าไหร่ ห่างจาก 0.5 มากน้อยแค่ไหน

เอา 10 อันดับที่ใกล้กับ 0.5 มากสุด
คือก็คือ ย้อนไปห้า เดินหน้าไปห้า ก็ได้เป็น
0.495 ถึง 0.505

ทำไมต้องเป็น 0.5
ก็คือระยะห่างระหว่าง 2 จุด
คือจุดว่า user คนนั้นให้ rating หนังเรื่องนี้เท่านี้ เต็ม 5.0
แล้ว user ของเราคนนี้จะให้เท่าไหร่

ยังไงก็ต้องมี user embedding ด้วย
เพราะมัน predict against person/identity

ก็คือให้ใส่ค่า rating ของหนังแต่ละเรื่องสำหรับคนๆนี้
ระหว่าง 0 กับ 1
0 คือไม่ดูแน่นอน เกลียดเลย
1 คือชอบเลิฟๆ ให้เต็ม
ก็คือ 1.0, 2.0 ถึง 5.0 rating นั่นเอง

In [ ]:
เสร็จก็เลยไปเพิ่มคอลัมน์ not_all_ratings['movie'] เพื่อจะได้เรียกหนังเรื่องนึงได้ ตาม index ของมัน
จะได้ลบออกได้
แล้วก็ใส่ not_all_ratings['user'] ด้วย
แต่มันมีอยู่แล้วนี่หว่า??
แต่มันเริ่มต้นที่ 0-609 เลยต้องไป encode รอบนึงก่อน? ไรเงี้ยเหรอ?

เสร็จก็เอา x = not_all_ratings[['user', 'movie']]
ใส่เข้าไปในโมเดลไปเทรน

y ก็คือ not_all_ratings['rating'] นั่นเองที่ให้ predict

ในโมเดลทำอะไร
ก็คำนวณ embedding, bias ออกมา แล้วก็ไปหาผล tf.tensordot
คูณกันออกมาเสร็จก็เอาไปใส่ฟังชั่น sigmoid ไม่รู้ทำไงนั่นแหละ tf.nn.sigmoid
คือแปลงให้อยู่ระหว่าง 0 กับ 1 ไอ้เคิร์ฟโค้งๆเอสนั่น

เป็นอันเสร็จพิธี

ต้องหาวิธีวัด performance ด้วยต่อไป

In [1]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import create_engine
import os
import pandas as pd

app = Flask(__name__)

engine = create_engine('sqlite:///recommender2.db', echo=False)

for f in os.listdir('data/movies/ml-latest-small'):
    if f[-4:] == '.csv':
        data = pd.read_csv(f'data/movies/ml-latest-small/{f}')
        data.to_sql(f[:-4], engine)
        print(f[0:-4])

links
ratings
movies
tags


In [2]:
engine

Engine(sqlite:///recommender2.db)

In [236]:
q = 'SHOW movies;'
movies_table = pd.read_sql(q, engine)
movies_table

OperationalError: (sqlite3.OperationalError) near "SHOW": syntax error
[SQL: SHOW movies;]
(Background on this error at: http://sqlalche.me/e/13/e3q8)

In [5]:
query = 'SELECT "userId", ratings."movieId", movies.title, rating FROM ratings JOIN movies ON ratings."movieId" = movies."movieId";'
all_ratings = pd.read_sql(query, engine)


In [6]:
all_ratings

,userId,movieId,title,rating
0,1,1,Toy Story (1995),4.0
1,1,3,Grumpier Old Men (1995),4.0
2,1,6,Heat (1995),4.0
3,1,47,Seven (a.k.a. Se7en) (1995),5.0
4,1,50,"Usual Suspects, The (1995)",5.0
...,...,...,...,...
100831,610,166534,Split (2017),4.0
100832,610,168248,John Wick: Chapter Two (2017),5.0
100833,610,168250,Get Out (2017),5.0
100834,610,168252,Logan (2017),5.0


In [ ]:
หน้าแรกโหลดช้า
บอกให้โหลดมาแค่ index.html ก่อน
แล้วค่อยมาโหลดdatabase ข้างหลังเป็น background process
ตอนที่ลูกค้ากำลังพิมพ์ชื่อหนังใส่เข้าไปอยู่ จะได้ไม่ต้องรอหน้าแรกนาน
คือหน้าแรกยังไม่ต้องโหลดดาต้าเบสมา ค่อยมาโหลดตอนรอ userinput ได้ไหม

In [4]:
watched_movie_id_list = ['70286', '109487', '589']


In [41]:
import numpy as np

movie_id_unique = 'SELECT * FROM movies'
all_movies = pd.read_sql(movie_id_unique, engine)

#remove the input movies from all_movies
movies_not_watched = all_movies[~all_movies['movieId'].isin(watched_movie_id_list)]
movies_not_watched.loc[:,'fake_id'] = np.ones(len(movies_not_watched), dtype =int)

#get all_ratings from sqlite
query = 'SELECT "userId", ratings."movieId", movies.title, rating FROM ratings JOIN movies ON ratings."movieId" = movies."movieId";'
all_ratings = pd.read_sql(query, engine)

#remove the watched movies from all_ratings
not_all_ratings = all_ratings[~all_ratings['movieId'].isin(watched_movie_id_list)]

#remove movieId and ratings of the watched movies???
movieindex = not_all_ratings['movieId'].unique().tolist()
dl_movie2movie_encoded = {x: i for i, x in enumerate(movieindex)}
dl_movie_encoded2movie = {i: x for i, x in enumerate(movieindex)}

not_all_ratings.loc[:,"movie"] = not_all_ratings["movieId"].map(dl_movie2movie_encoded)
not_all_ratings.loc[:,"rating"] = not_all_ratings["rating"].values.astype(np.float32)

#map userId from ?? to not_all_ratings ?? what do i do with the users??

not_all_user_ids = not_all_ratings["userId"].unique().tolist()
dl_user2user_encoded = {x: i for i, x in enumerate(not_all_user_ids)}
dl_userencoded2user = {i: x for i, x in enumerate(not_all_user_ids)}

not_all_ratings.loc[:,"user"] = not_all_ratings["userId"].map(dl_user2user_encoded)

min_rating = min(not_all_ratings["rating"])
max_rating = max(not_all_ratings["rating"])
num_users = len(dl_user2user_encoded)
num_movies = len(dl_movie_encoded2movie)
print(
        "Number of users: {}, Number of Movies: {}, Min rating: {}, Max rating: {}".format(
            num_users, num_movies, min_rating, max_rating
        )
    )

#define training data
df = not_all_ratings.sample(frac=1, random_state=42)
x = not_all_ratings[["user", "movie"]].values

    # Normalize the targets between 0 and 1. Makes it easy to train.
y = not_all_ratings["rating"].apply(lambda x: (x - min_rating) / (max_rating - min_rating)).values
    # Assuming training on 90% of the data and validating on 10%.
train_indices = int(0.9 * df.shape[0])
x_train, x_val, y_train, y_val = (
        x[:train_indices],
        x[train_indices:],
        y[:train_indices],
        y[train_indices:],
    )

#inference
user_movie_array = movies_not_watched[['fake_id','index']]
max_movie_index = user_movie_array['index'].max()
that = user_movie_array.to_numpy()

EMBEDDING_SIZE = 50

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class RecommenderNet(keras.Model):
        def __init__(self, num_users, num_movies, embedding_size, **kwargs):
            super(RecommenderNet, self).__init__(**kwargs)
            self.num_users = num_users
            self.num_movies = num_movies
            self.embedding_size = embedding_size
            self.user_embedding = layers.Embedding(
                num_users,
                embedding_size,
                embeddings_initializer="he_normal",
                embeddings_regularizer=keras.regularizers.l2(1e-6),
            )
            self.user_bias = layers.Embedding(num_users, 1)
            self.movie_embedding = layers.Embedding(
            #change this line from num_movies to max_movie_index+1
                max_movie_index+1,
                embedding_size,
                embeddings_initializer="he_normal",
                embeddings_regularizer=keras.regularizers.l2(1e-6),
            )
            #and this line (input_dim)
            self.movie_bias = layers.Embedding(max_movie_index+1, 1)

        def call(self, inputs):
            user_vector = self.user_embedding(inputs[:, 0])
            user_bias = self.user_bias(inputs[:, 0])
            movie_vector = self.movie_embedding(inputs[:, 1])
            movie_bias = self.movie_bias(inputs[:, 1])
            dot_user_movie = tf.tensordot(user_vector, movie_vector, 2)
            # Add all the components (including bias)
            x = dot_user_movie + user_bias + movie_bias
            # The sigmoid activation forces the rating to between 0 and 1
            return tf.nn.sigmoid(x)
            #return movie_bias

model = RecommenderNet(num_users, num_movies, EMBEDDING_SIZE)
model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(), optimizer=keras.optimizers.Adam(lr=0.001)
    )
ratings = model.predict(that).flatten()

top_ratings_indices = ratings.argsort()[-10:][::-1]
movies_not_watched.loc[movies_not_watched['index'].isin(top_ratings_indices)]

/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


Number of users: 610, Number of Movies: 9721, Min rating: 0.5, Max rating: 5.0


,index,movieId,title,genres,fake_id
1941,1941,2573,Tango (1998),Drama|Musical,1
7042,7042,69118,In the Electric Mist (2009),Drama|Mystery,1
7049,7049,69227,Ernest Rides Again (1993),Children|Comedy,1
7053,7053,69304,Imagine That (2009),Comedy|Drama|Fantasy,1
7054,7054,69306,"Taking of Pelham 1 2 3, The (2009)",Crime|Drama|Thriller,1
7055,7055,69394,"Stoning of Soraya M., The (2008)",Crime|Drama,1
7059,7059,69469,Garfield's Pet Force (2009),Animation,1
7061,7061,69495,Breakfast with Scot (2007),Drama|Romance,1
7065,7065,69529,Home (2009),Documentary,1
7070,7070,69654,Prison Break: The Final Break (2009),Action|Drama|Thriller,1


In [27]:
#จะเรียก movie_bias มาดู ยังเNameError: name 'inputs' is not definedรียกไม่เป้น
recommender = RecommenderNet(int('num_users', 'num_movies', 'embedding_size'))
recommender.call()

TypeError: int() takes at most 2 arguments (3 given)

In [43]:
recommender.call(inputs)

NameError: name 'inputs' is not defined

In [14]:
num_users

610

In [15]:
num_movies

9721

In [18]:
EMBEDDING_SIZE

50

In [28]:
movie_bias

NameError: name 'movie_bias' is not defined

In [30]:
x

array([[   0,    0],
       [   0,    1],
       [   0,    2],
       ...,
       [ 609, 3118],
       [ 609, 1389],
       [ 609, 2870]])

In [31]:
x.shape

(100474, 2)

In [ ]:
1 Feb 2021 เปิดโปรเจ็คใหม่ หลังจาก deploy secret-beach-84256.herokuapp.com เรียบร้อยแล้ว
ก็เกิดอยากจะมาแก้งานอัลกอ ที่เป็นสมองของ recommender ว่าอยากให้
taking into account more of ['genres'] column
เพิ่ม bias weight ให้ genres หนักๆหน่อย น่าจะ recommend ได้ดีขึ้น

ก็เลยเป็นที่มาของการกลับมาทำต่อตอนนี้ เพราะม่ำบอกว่า ใช้อีกนาน ไม่ต้องรีบ
คือตอบคำถามที่เราถามไปว่า สกิลการอีดิทโค้ด keras มันคงได้ใช้อีกใช่ไม๊
ซึ่งตอบว่าใช้อีกนานนี่คือสกิล หรือว่า งานของมันอีกนานกว่าจะใช้

มาอ่านดู จริงๆแค่กลับไปใช้ content-based ก็ได้
แต่เราอยากให้เอา column ['genres'] ไปคำนวณด้วยต่างหาก
เริ่มแรก ก็ต้อง EDA ดูก่อนว่าคอลัมน์นี้มีอะไรบ้าง

แล้วก็ไปดึงคอลัมน์นี้มาจาก watched_movies_list ที่ userinput เข้ามา

จากนั้นก็เอาคอลัมน์นี้ใส่เข้าไปในเลเยอร์ ตรงไหนก็ไม่รู้ล่ะ 
หรือจะเป็นส่วนหนึ่งของ movie_bias อันนี้ต้องไปอ่านละเอียดอีกที

คือแค่ดึงอัลกอ deep learning ทิ้งทั้งยวงแล้วใส่โมเดลอีกอันไป ซึ่งคือ content-based filtering
มีโค้ดแค่ไม่กี่บรรทัด

ปรากฏว่า genre ก็มีค่าเยอะมาก โดยใส่ | ระหว่างกันไว้ถ้ามีหลาย tag
เช่น ถ้าอยากได้หนังไซไฟ จะบอกแค่ Sci-fi อย่างเดียวไม่พอ เพราะในลิสท์มีหนังที่ระบุ genres Sci-fi มีแค่ 37 เรื่อง

แต่มี Sci-Fi|IMAX,Mystery|Sci-Fi|Thriller, Action|Sci-fi

ฉะนั้น ก็ต้องทำเป็น list
ให้หาคำไหนก้ได้ ในคำไหนก็ได้เฉพาะในลิสท์นี้ genres อื่นตัดทิ้งไปเลย ฉะนั้นพวก drama, comedy, romance ลบออกไปเลย
ก็จะเหลือแค่ unique_genres = {Sci-fi, IMAX, Mystery, Thriller, Action}
แต่อย่างงี้ sci-fi ก็จะไม่ได้รับการเน้นอย่างที่ควรจะเป็น ควรจะต้องนับไปด้วย
แต่มี Sci-Fi|IMAX,Mystery|Sci-Fi|Thriller, Action|Sci-fi

ทำยังไงถึงจะให้มันรู้ว่าควรต้องเลือก Sci-fi ก่อน แต่ไม่ใช่เอามาแต่ Sci-fi อย่างเดียวนะ ต้องอันที่มีส่วนประกอบด้วยอย่างอื่น
หาว่าแต่ละelement มี intersect กันไหม ถ้ามีที่ A ^ B ^ C เป็น 1 ถ้ามีก็เอาอันนั้น
คำที่สองรองมาก็เช่นมี A ^ B, หรือ A^C, B^C ทั้งสามคู่ หรือมีสองคู่ หรือมีคู่เดียว
ขึ้นอยู่กับว่า userinput ใส่มากี่อัน ก็เท่านั้นคู่ นี่มันเป็นเรื่อง probability อีกแล้วหนิ
n! n-1 อะไรเนี่ย
แล้วก็พวกที่มีค่าแค่ครั้งเดียว ก็เอามาต่อท้ายตารางด้วย IMAX, Mystery, Thriller, Action

list values ทั้งหมดของ ['genres'] ออกมาดู ไม่ใช่มากดแอพไล่หาทีละเรื่องก็ไม่ได้

Wicked City (Yôjû toshi) (1987) Animation|Fantasy|Horror|Sci-Fi
Legion (2010) Action|Fantasy|Horror|Thriller

สองเรื่องนี้มาได้ แต่ที่เหลือ comedy romance drama มันไม่ใช่เลย fantasy ยังไม่ได้เลย
เรื่องที่สองคะแนนจะได้น้อยหน่อย เพราะมี Actionกับ Thriller แต่ไม่มี Sci-Fi ซึ่งได้คะแนนเต็ม

เวลาคิดคะแนนก็ต้องเอา หนังเรื่องที่มี genres อันที่ intersect ครบสามก่อน
มาคิดคะแนน genres_bias ของหนังแต่ละเรื่อง
แล้วค่อยนำไปคำนวณคะแนน รวมกับ? ไม่ หรือว่าเอาแค่นี้แหละไปคิดคะแนน ก็แค่คูณ vector
หาอันที่ได้คะแนนสูงสุด แล้วก็ mask ชื่อหนังกลับมาแค่นั้นแหละ
ไม่ได้เกี่ยวกับ ratings อื่น ของคนอื่นเลย?

ไม่ๆ ต้องเอาไปรวมกับผลคะแนนที่คนอื่นๆเรทให้ด้วย แต่ต้องเฉพาะหนังตามลิสท์นี้เท่านั้นนะ
แล้วค่อยหา predict มาตอนท้าย ที่ออกมามากกว่า 0.51 - 0.54 ประมาณนี้ 0.526254
0.526094 0.526847
นี่แค่ genres_bias ok ok

แล้วเอาหนังที่ไม่มี genres ตามในลิสท์ ลบออกจาก movies_not_watched ที่จะเอามาใส่ตอนพรีดิกด้วยนะ


In [23]:
not_all_ratings

,userId,movieId,title,rating,movie,user
0,1,1,Toy Story (1995),4.0,0,0
1,1,3,Grumpier Old Men (1995),4.0,1,0
2,1,6,Heat (1995),4.0,2,0
3,1,47,Seven (a.k.a. Se7en) (1995),5.0,3,0
4,1,50,"Usual Suspects, The (1995)",5.0,4,0
...,...,...,...,...,...,...
100831,610,166534,Split (2017),4.0,3117,609
100832,610,168248,John Wick: Chapter Two (2017),5.0,2032,609
100833,610,168250,Get Out (2017),5.0,3118,609
100834,610,168252,Logan (2017),5.0,1389,609


In [24]:
all_movies

,index,movieId,title,genres
0,0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,2,Jumanji (1995),Adventure|Children|Fantasy
2,2,3,Grumpier Old Men (1995),Comedy|Romance
3,3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...,...
9737,9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,9739,193585,Flint (2017),Drama
9740,9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [26]:
all_movies[all_movies['genres'] == 'Sci-Fi']

,index,movieId,title,genres
1719,1719,2311,2010: The Year We Make Contact (1984),Sci-Fi
1902,1902,2526,Meteor (1979),Sci-Fi
2000,2000,2661,It Came from Outer Space (1953),Sci-Fi
2003,2003,2665,Earth vs. the Flying Saucers (1956),Sci-Fi
2026,2026,2698,Zone 39 (1997),Sci-Fi
2507,2507,3354,Mission to Mars (2000),Sci-Fi
3121,3121,4198,Battle Beyond the Stars (1980),Sci-Fi
3522,3522,4813,When Worlds Collide (1951),Sci-Fi
3600,3600,4942,"Angry Red Planet, The (1959)",Sci-Fi
3893,3893,5468,20 Million Miles to Earth (1957),Sci-Fi


In [27]:
len(all_movies[all_movies['genres'] == 'Sci-Fi'])

37

In [45]:
not_all_ratings.loc[not_all_ratings['title']=='Arrival(2016)']

,userId,movieId,title,rating,movie,user


In [47]:
all_movies.loc[all_movies['title'] == 'Interstellar (2014)']

,index,movieId,title,genres
8376,8376,109487,Interstellar (2014),Sci-Fi|IMAX


In [100]:
watched_movie_id_list = ['70286', '109487', '589']


In [62]:
all_movies.loc[all_movies['movieId'] == 109487]['genres']

8376    Sci-Fi|IMAX
Name: genres, dtype: object

In [63]:
all_movies.loc[all_movies['movieId'] == 70286]['genres']

7090    Mystery|Sci-Fi|Thriller
Name: genres, dtype: object

In [64]:
all_movies.loc[all_movies['movieId'] == 589]['genres']

507    Action|Sci-Fi
Name: genres, dtype: object

In [102]:
watched_movie_id_list = ['70286', '109487', '589']

movies_genres_user_likes = []

for i in watched_movie_id_list:
    liked_genres = all_movies.loc[all_movies['movieId'] == i]['genres']
    print(liked_genres)
    movies_genres_user_likes.append(liked_genres)
    
#แค่โค้ดบรรทัดแรกก็จอดละ

Series([], Name: genres, dtype: object)
Series([], Name: genres, dtype: object)
Series([], Name: genres, dtype: object)


In [101]:
print(movies_genres_user_likes.append(liked_genres))

None


In [121]:
movies_genres_user_likes

[Series([], Name: genres, dtype: object),
 Series([], Name: genres, dtype: object),
 Series([], Name: genres, dtype: object)]

In [123]:
liked_genres

Series([], Name: genres, dtype: object)

In [110]:
i = 70286
all_movies.loc[all_movies['movieId'] == i]['genres']

7090    Mystery|Sci-Fi|Thriller
Name: genres, dtype: object

In [ ]:
languages = ['Spanish', 'English',  'French', 'German', 'Irish', 'Chinese']

for index in range(len(languages)):
   print('Language:', languages[index])

In [ ]:
enumerate

In [151]:
i = ['70286', '109487']
languages = ['Spanish', 'English',  'French', 'German', 'Irish', 'Chinese']

for index in range(len(languages)):
   print('Language:', languages[index])

for index in range(len(i)):
    print('movieId:', i[index])

#ValueError: ('Lengths must match to compare', (9742,), (2,))
#งั้นต้องทำให้มันความยาวเท่ากันเหรอ?


Language: Spanish
Language: English
Language: French
Language: German
Language: Irish
Language: Chinese
movieId: 70286
movieId: 109487


In [152]:
for index in range(len(i)):
    print(all_movies.loc[all_movies['movieId'] == i[index]]['genres'])


Series([], Name: genres, dtype: object)
Series([], Name: genres, dtype: object)


In [153]:
i[index]

'109487'

In [156]:
all_movies.loc[all_movies['movieId'] == i[index]]['genres']

Series([], Name: genres, dtype: object)

In [141]:
i = [70286, 109487]
for i in i:
    all_movies.loc[all_movies.movieId == 980, 'genres']
    #all_movies.loc[all_movies.movieId == 109487, 'genres'] 
    print(all_movies.loc[all_movies.movieId == 980, 'genres'])

Series([], Name: genres, dtype: object)
Series([], Name: genres, dtype: object)


In [120]:
get the genres from all the items in watched_movie_id_list

NameError: name 'Series' is not defined

In [162]:
i = [70286, 109487, 589]

In [ ]:
watched_movie_id_list = ['70286', '109487', '589']


In [159]:
i = ['70286', '109487']

In [160]:
##here it works

all_movies.loc[all_movies['movieId'].isin(i), 'genres']
#df.loc[df['column_name'].isin(some_values)]


7090    Mystery|Sci-Fi|Thriller
8376                Sci-Fi|IMAX
Name: genres, dtype: object

In [163]:
#this lilne is what i need first thing

all_movies.loc[all_movies['movieId'].isin(i), 'genres'].to_list()

['Action|Sci-Fi', 'Mystery|Sci-Fi|Thriller', 'Sci-Fi|IMAX']

In [167]:
liked_genres = all_movies.loc[all_movies['movieId'].isin(i), 'genres'].to_list()
liked_genres

['Action|Sci-Fi', 'Mystery|Sci-Fi|Thriller', 'Sci-Fi|IMAX']

In [ ]:
#now, how can i seperate Action from Sci-fi?? becaues I have to find the intersection of Sci-fi 
#in the 3 elements
#first have to separate whatever inside the element with |
#then look at each element if there's intersection

In [165]:
all_movies.loc[all_movies['movieId'].isin(i), 'genres'].to_dict()

{507: 'Action|Sci-Fi', 7090: 'Mystery|Sci-Fi|Thriller', 8376: 'Sci-Fi|IMAX'}

In [169]:
df_genres = pd.DataFrame([sub.split("|") for sub in liked_genres])
df_genres
#df = pd.DataFrame([sub.split(",") for sub in l])

,0,1,2
0,Action,Sci-Fi,None
1,Mystery,Sci-Fi,Thriller
2,Sci-Fi,IMAX,None


In [ ]:
remove None? no need? it's not Nan?

find if any value is in all rows
find if any value is in at least 2 rows


In [173]:
findscifi = df_genres == "Sci-Fi"
findscifi

,0,1,2
0,False,True,False
1,False,True,False
2,True,False,False


In [ ]:
your_df == “some_value”` : 
This will create a boolean mask of your whole DataFrame. 
If you want to search for directly in a column, use `df[‘your_column_name’]`. 
If you want to search on multiple columns, do `df[ [‘column_1’, ‘column_n’] ]`. 
For indexes, use either `df.loc[]` or `df.iloc[]`.
Having a boolean array, you shall now use the method `.any()` on your newly created DataFrame, 
such as : `masked_df.any()`. 
    This will return if any on the whole DataFrame has matched. 
But you can also specify if you want to make a search by column (`.any(axis=1)`), 
or row (`.any(axis=0)`).

In [178]:
findscifi.any(axis=1)

0    True
1    True
2    True
dtype: bool

In [180]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = ['This is the first document.', 'This document is the second document.','And this is the third one.','Is this the first document?']

In [181]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

In [182]:
print(vectorizer.get_feature_names())

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [ ]:
print(vectorizer.get_feature_names())
['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']

In [183]:
print(X.toarray())

[[0 1 1 1 0 0 1 0 1]
 [0 2 0 1 0 1 1 0 1]
 [1 0 0 1 1 0 1 1 1]
 [0 1 1 1 0 0 1 0 1]]


In [231]:
liked_genres

['Action|Sci-Fi', 'Mystery|Sci-Fi|Thriller', 'Sci-Fi|IMAX']

In [186]:
LG = vectorizer.fit_transform(liked_genres)

In [187]:
print(vectorizer.get_feature_names())

['action', 'fi', 'imax', 'mystery', 'sci', 'thriller']


In [188]:
print(LG.toarray())

[[1 1 0 0 1 0]
 [0 1 0 1 1 1]
 [0 1 1 0 1 0]]


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
>>> corpus = [
...     'This is the first document.',
...     'This document is the second document.',
...     'And this is the third one.',
...     'Is this the first document?',
... ]
>>> vectorizer = TfidfVectorizer()
>>> X = vectorizer.fit_transform(corpus)
>>> print(vectorizer.get_feature_names())
['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']
>>> print(X.shape)
(4, 9)

In [232]:
from sklearn.feature_extraction.text import TfidfVectorize
liked_genres = ['Action|Sci-Fi', 'Mystery|Sci-Fi|Thriller', 'Sci-Fi|IMAX']

vectorizer = TfidfVectorizer()
Ha = vectorizer.fit_transform(liked_genres)


In [234]:
print(vectorizer.get_feature_names())
print(Ha.shape)

['action', 'fi', 'imax', 'mystery', 'sci', 'thriller']
(3, 6)


In [ ]:
ตอนนี้อยากเห็นไร
content-based filtering ชัดๆ

อยากได้ genres bias 
เอา movies_not_watched มา ลบเรื่องที่ไม่มี sci-fi, action, IMAX, mystery, thriller ออก
ส่วนเรื่องที่ยังอยู่ ถ้ามี sci-fi ให้สามคะแนน
ถ้ามีอีกคำนึงนี่มีอยู่ใน 2 rows เช่น ในกรณีนี้คือมี None ก็เอาไปอีกสองคะแนน
ส่วนเรื่องไหนมี action หรือ mystery หรือ thriller ก็เอาไปอีกอันละ 0.5 
คือหนังที่เป็น Action|Mystery|Thriller เต็มๆ โดยไม่มี Sci-fi จะได้ 1.5 คะแนน น้อยกว่า sci-fi เปล่าๆอยู่ดี
ถ้าเป็น Mystery|Sci-fi|Thriller งี้จะได้เยอะเลย 4 คะแนน

แต่มันต้องมีคนคิดมาแล้วสิวะแบบนี้


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(movies['genres'])

from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

#I now have a pairwise cosine similarity matrix for all the movies in the dataset. 
#The next step is to write a function that returns the 20 most similar movies based on the cosine similarity score.

# Build a 1-dimensional array with movie titles
titles = movies['title']
indices = pd.Series(movies.index, index=movies['title'])

# Function that get movie recommendations based on the cosine similarity score of movie genres

def genre_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [190]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(all_movies['genres'])

from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

#I now have a pairwise cosine similarity matrix for all the movies in the dataset. 
#The next step is to write a function that returns the 20 most similar movies based on the cosine similarity score.

# Build a 1-dimensional array with movie titles
titles = all_movies['title']
indices = pd.Series(all_movies.index, index=all_movies['title'])

# Function that get movie recommendations based on the cosine similarity score of movie genres

def genre_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [196]:
genre_recommendations('Interstellar (2014)')

8013                                   Cloud Atlas (2012)
8414                                 Transcendence (2014)
8252                                       Gravity (2013)
8406                      The Amazing Spider-Man 2 (2014)
8438                              Edge of Tomorrow (2014)
3832    Star Wars: Episode II - Attack of the Clones (...
5260                                  Spider-Man 2 (2004)
6238                              Superman Returns (2006)
7018                                     Star Trek (2009)
7064           Transformers: Revenge of the Fallen (2009)
7212                                        Avatar (2009)
7484                                  Tron: Legacy (2010)
7693                                 Avengers, The (2012)
7842                                   John Carter (2012)
7927                       Amazing Spider-Man, The (2012)
8137                                      Oblivion (2013)
8159                       Star Trek Into Darkness (2013)
8178          

In [216]:
tf

TfidfVectorizer(min_df=0, ngram_range=(1, 2), stop_words='english')

In [ ]:
อยากได้ TfIDFVectorizer ระหว่างหนังหลายๆเรื่อง
#TfidfVector in amongst many movies? แล้วจะหาได้ยังไง
เอา cosine_sim มาทำอะไรได้บ้าง ใส่เข้าไปใน layers ได้ไหม

In [210]:
tfidf_matrix

<9742x177 sparse matrix of type '<class 'numpy.float64'>'
	with 36628 stored elements in Compressed Sparse Row format>

In [211]:
cosine_sim

array([[1.        , 0.31379419, 0.0611029 , ..., 0.        , 0.16123168,
        0.16761358],
       [0.31379419, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.0611029 , 0.        , 1.        , ..., 0.        , 0.        ,
        0.36454626],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.16123168, 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.16761358, 0.        , 0.36454626, ..., 0.        , 0.        ,
        1.        ]])

In [212]:
cosine_sim.shape

(9742, 9742)

In [213]:
cosine_sim[0:5]

array([[1.        , 0.31379419, 0.0611029 , ..., 0.        , 0.16123168,
        0.16761358],
       [0.31379419, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.0611029 , 0.        , 1.        , ..., 0.        , 0.        ,
        0.36454626],
       [0.05271111, 0.        , 0.35172407, ..., 0.29043659, 0.        ,
        0.31447995],
       [0.16761358, 0.        , 0.36454626, ..., 0.        , 0.        ,
        1.        ]])

In [225]:
cosine_sim[3].mean()

0.2087052599180262

In [226]:
cosine_sim[3].mean

<function ndarray.mean>

In [227]:
cosine_sim[27].mean()

0.1530538364306775

In [230]:
cosine_sim[1230].mean()

0.20259926562452402

In [229]:
cosine_sim[9000].mean()

0.06838394506703996

In [209]:
indices

title
Toy Story (1995)                                0
Jumanji (1995)                                  1
Grumpier Old Men (1995)                         2
Waiting to Exhale (1995)                        3
Father of the Bride Part II (1995)              4
                                             ... 
Black Butler: Book of the Atlantic (2017)    9737
No Game No Life: Zero (2017)                 9738
Flint (2017)                                 9739
Bungo Stray Dogs: Dead Apple (2018)          9740
Andrew Dice Clay: Dice Rules (1991)          9741
Length: 9742, dtype: int64

In [ ]:
8013                                   Cloud Atlas (2012)

In [201]:
all_movies.loc[all_movies['title'] == 'Cloud Atlas (2012)']['genres']

8013    Drama|Sci-Fi|IMAX
Name: genres, dtype: object

In [202]:
all_movies.loc[all_movies['title'] == 'Transcendence (2014)']['genres']

8414    Drama|Sci-Fi|IMAX
Name: genres, dtype: object

In [203]:
all_movies.loc[all_movies['title'] == 'Oblivion (2013)']['genres']

8137    Action|Adventure|Sci-Fi|IMAX
Name: genres, dtype: object

In [204]:
all_movies.loc[all_movies['title'] == 'John Carter (2012)']['genres']

7842    Action|Adventure|Sci-Fi|IMAX
Name: genres, dtype: object

In [ ]:
อือ นี่มันสำหรับใส่หนังเรื่องเดียวเลยนะ
ถ้าuser ใส่เรื่องเดียวแบบนี้ก็โอเค แต่ถ้าหลายเรื่องก็หลายชั้นอีก

In [ ]:
Spider-Man 2 (2004)

In [205]:
all_movies.loc[all_movies['title'] == 'Spider-Man 2 (2004)']['genres']

5260    Action|Adventure|Sci-Fi|IMAX
Name: genres, dtype: object

In [206]:
all_movies.loc[all_movies['title'] == 'Star Trek (2009)']['genres']

7018    Action|Adventure|Sci-Fi|IMAX
Name: genres, dtype: object

In [208]:
all_movies.loc[all_movies['title'] == 'Avengers, The (2012)']['genres']

7693    Action|Adventure|Sci-Fi|IMAX
Name: genres, dtype: object

In [ ]:
จริงๆแล้วเราไม่ได้ชอบ interstellar ด้วยความ IMAX สักนิด
เราชอบด้วยความเรียลของเนื้อหา ความยากในการตามถ้าไม่อ่านมาก่อน ความวิทยาศาสตร์
ไม่ได้ชอบ Action|Adventure ขนาดนั้น 
ในเนื้อเรื่องความสัมพันธ์พ่อลูกไรนี่ก็ชอบ

Cloud Atlas, TRON:Legacy, Spider Man นี่ก็ได้
Transcendence น่าจะชอบ แค่ฟังชื่อ Oblivion, After Earth
แต่ Star Trek, Star Wars, Superman นี่ไม่ได้ โหลเกินไป

In [ ]:
ถ้าเราไม่ต้องเอาทั้งหมดมา intersect กัน
เราแค่ให้น้ำหนักของหนังเรื่องที่ genres เดียวกันมากขึ้นหน่อยนึง 
มันจะ make it into the top ten ป่ะวะ

In [ ]:
Sci-Fi|IMAX'

In [220]:
all_movies[all_movies['genres'] == 'Sci-Fi|IMAX'] #มีเรื่องเดียวเลยจ้า

,index,movieId,title,genres
8376,8376,109487,Interstellar (2014),Sci-Fi|IMAX


In [ ]:
7090    Mystery|Sci-Fi|Thriller

In [221]:
all_movies[all_movies['genres'] == 'Mystery|Sci-Fi|Thriller'] 

,index,movieId,title,genres
31,31,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
91,91,103,Unforgettable (1996),Mystery|Sci-Fi|Thriller
1748,1748,2346,"Stepford Wives, The (1975)",Mystery|Sci-Fi|Thriller
3674,3674,5062,Seconds (1966),Mystery|Sci-Fi|Thriller
7090,7090,70286,District 9 (2009),Mystery|Sci-Fi|Thriller
9085,9085,143255,Narcopolis (2014),Mystery|Sci-Fi|Thriller


In [117]:
#df.loc[df.Name=='Pigeon','Category'].iteritems()
#all_movies.loc[all_movies['movieId'] == i]['genres']
all_movies.loc[all_movies.movieId == 109487, 'genres'].iteritems()

In [ ]:
vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(2, 2))
X2 = vectorizer2.fit_transform(corpus)
print(vectorizer2.get_feature_names())
['and this', 'document is', 'first 

In [115]:
#df.loc[df.Name=='Pigeon','Category'].index.values

all_movies.loc[all_movies.movieId == 109487]['genres'].index.values

array([8376])

In [65]:
all_movies[all_movies['genres'] == 'Action|Sci-Fi']

,index,movieId,title,genres
507,507,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi
595,595,737,Barb Wire (1996),Action|Sci-Fi
1056,1056,1373,Star Trek V: The Final Frontier (1989),Action|Sci-Fi
1261,1261,1676,Starship Troopers (1997),Action|Sci-Fi
1767,1767,2365,King Kong vs. Godzilla (Kingukongu tai Gojira)...,Action|Sci-Fi
1906,1906,2530,Beneath the Planet of the Apes (1970),Action|Sci-Fi
1907,1907,2531,Battle for the Planet of the Apes (1973),Action|Sci-Fi
1908,1908,2532,Conquest of the Planet of the Apes (1972),Action|Sci-Fi
1909,1909,2533,Escape from the Planet of the Apes (1971),Action|Sci-Fi
1921,1921,2549,Wing Commander (1999),Action|Sci-Fi


In [67]:
all_movies[all_movies['genres'] == 'Sci-Fi|Thriller']

,index,movieId,title,genres
668,668,880,"Island of Dr. Moreau, The (1996)",Sci-Fi|Thriller
1320,1320,1779,Sphere (1998),Sci-Fi|Thriller
2598,2598,3474,Retroactive (1997),Sci-Fi|Thriller
2934,2934,3937,Runaway (1984),Sci-Fi|Thriller
3617,3617,4966,"Incredible Shrinking Man, The (1957)",Sci-Fi|Thriller
3670,3670,5054,Brainstorm (1983),Sci-Fi|Thriller
3942,3942,5544,Time After Time (1979),Sci-Fi|Thriller
5360,5360,8939,"Final Cut, The (2004)",Sci-Fi|Thriller
5496,5496,26365,Futureworld (1976),Sci-Fi|Thriller
6954,6954,65642,"Timecrimes (Cronocrímenes, Los) (2007)",Sci-Fi|Thriller


In [68]:
all_movies.loc[all_movies['title'] == 'District 9 (2009)']

,index,movieId,title,genres
7090,7090,70286,District 9 (2009),Mystery|Sci-Fi|Thriller


In [69]:
all_movies[all_movies['genres'] == 'Mystery|Sci-Fi|Thriller']

,index,movieId,title,genres
31,31,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
91,91,103,Unforgettable (1996),Mystery|Sci-Fi|Thriller
1748,1748,2346,"Stepford Wives, The (1975)",Mystery|Sci-Fi|Thriller
3674,3674,5062,Seconds (1966),Mystery|Sci-Fi|Thriller
7090,7090,70286,District 9 (2009),Mystery|Sci-Fi|Thriller
9085,9085,143255,Narcopolis (2014),Mystery|Sci-Fi|Thriller


In [ ]:
เอาคำพวกนี้มาทำเป็นดิก

In [72]:
all_movies['genres'].values

array(['Adventure|Animation|Children|Comedy|Fantasy',
       'Adventure|Children|Fantasy', 'Comedy|Romance', ..., 'Drama',
       'Action|Animation', 'Comedy'], dtype=object)

In [78]:
all_movies['genres'].values[:200]

array(['Adventure|Animation|Children|Comedy|Fantasy',
       'Adventure|Children|Fantasy', 'Comedy|Romance',
       'Comedy|Drama|Romance', 'Comedy', 'Action|Crime|Thriller',
       'Comedy|Romance', 'Adventure|Children', 'Action',
       'Action|Adventure|Thriller', 'Comedy|Drama|Romance',
       'Comedy|Horror', 'Adventure|Animation|Children', 'Drama',
       'Action|Adventure|Romance', 'Crime|Drama', 'Drama|Romance',
       'Comedy', 'Comedy', 'Action|Comedy|Crime|Drama|Thriller',
       'Comedy|Crime|Thriller', 'Crime|Drama|Horror|Mystery|Thriller',
       'Action|Crime|Thriller', 'Drama|Sci-Fi', 'Drama|Romance', 'Drama',
       'Children|Drama', 'Drama|Romance',
       'Adventure|Drama|Fantasy|Mystery|Sci-Fi', 'Crime|Drama', 'Drama',
       'Mystery|Sci-Fi|Thriller', 'Children|Drama', 'Crime|Drama',
       'Children|Comedy', 'Comedy|Romance', 'Drama', 'Drama|War',
       'Action|Crime|Drama', 'Drama', 'Action|Adventure|Fantasy',
       'Comedy|Drama|Thriller', 'Drama|Romance', 'My

In [79]:
len(all_movies['genres'].values)

9742

In [81]:
np.unique(all_movies['genres'].values)

array(['(no genres listed)', 'Action', 'Action|Adventure',
       'Action|Adventure|Animation',
       'Action|Adventure|Animation|Children',
       'Action|Adventure|Animation|Children|Comedy',
       'Action|Adventure|Animation|Children|Comedy|Fantasy',
       'Action|Adventure|Animation|Children|Comedy|IMAX',
       'Action|Adventure|Animation|Children|Comedy|Romance',
       'Action|Adventure|Animation|Children|Comedy|Sci-Fi',
       'Action|Adventure|Animation|Children|Comedy|Sci-Fi|IMAX',
       'Action|Adventure|Animation|Children|Comedy|Western',
       'Action|Adventure|Animation|Children|Fantasy',
       'Action|Adventure|Animation|Children|Fantasy|Sci-Fi',
       'Action|Adventure|Animation|Children|Sci-Fi',
       'Action|Adventure|Animation|Comedy',
       'Action|Adventure|Animation|Comedy|Crime|Mystery',
       'Action|Adventure|Animation|Comedy|Fantasy|Mystery|Sci-Fi',
       'Action|Adventure|Animation|Comedy|Fantasy|Sci-Fi',
       'Action|Adventure|Animation|Comedy|S

In [82]:
len(np.unique(all_movies['genres'].values))

951

In [217]:
all_movies['genres']

0       Adventure|Animation|Children|Comedy|Fantasy
1                        Adventure|Children|Fantasy
2                                    Comedy|Romance
3                              Comedy|Drama|Romance
4                                            Comedy
                           ...                     
9737                Action|Animation|Comedy|Fantasy
9738                       Animation|Comedy|Fantasy
9739                                          Drama
9740                               Action|Animation
9741                                         Comedy
Name: genres, Length: 9742, dtype: object

In [ ]:
ณ ตอนนี้ มีอันที่น่าจะโดนลบออกเยอะมาก คือ 6 จาก 10 และ 8 จาก 10 ไม่ใช่ๆ userinput ไม่เหมือนกัน! 
คือโอกาสจะชอบต่ำมาก คิดมาได้ไงวะ machine อะไรของมึงโง่จริงแค่นี้ยังไม่รู้

In [ ]:
https://secret-beach-84256.herokuapp.com/recommend?
    movie2=Star+Wars%3A+Episode+I+-+The+Phantom+Menace+%281999%29&
    movie3=Interstellar+%282014%29&
    movie4=Armageddon+%281998%29

In [ ]:
Star Wars Episode I: The Phantom Menace


In [94]:
all_movies.loc[all_movies['title'] == 'Star Wars: Episode I - The Phantom Menace (1999)']

,index,movieId,title,genres
1979,1979,2628,Star Wars: Episode I - The Phantom Menace (1999),Action|Adventure|Sci-Fi


In [95]:
all_movies.loc[all_movies['title'] == 'Donnie Darko (2001)']

,index,movieId,title,genres
3562,3562,4878,Donnie Darko (2001),Drama|Mystery|Sci-Fi|Thriller


In [87]:
all_movies.loc[all_movies['title'] == 'Armageddon (1998)']

,index,movieId,title,genres
1398,1398,1917,Armageddon (1998),Action|Romance|Sci-Fi|Thriller


In [ ]:
Comedy|Fantasy|Sci-Fi',

In [218]:
all_movies.loc[all_movies['genres'] == 'Comedy|Fantasy|Sci-Fi']

,index,movieId,title,genres
583,583,718,"Visitors, The (Visiteurs, Les) (1993)",Comedy|Fantasy|Sci-Fi
1592,1592,2134,Weird Science (1985),Comedy|Fantasy|Sci-Fi
2039,2039,2717,Ghostbusters II (1989),Comedy|Fantasy|Sci-Fi
9418,9418,165639,While You Were Fighting: A Thor Mockumentary (...,Comedy|Fantasy|Sci-Fi
9712,9712,188189,Sorry to Bother You (2018),Comedy|Fantasy|Sci-Fi


In [ ]:
       'Action|Animation|Crime|Drama|Film-Noir|Mystery|Sci-Fi|Thriller',

In [219]:
all_movies.loc[all_movies['genres'] == 'Action|Animation|Crime|Drama|Film-Noir|Mystery|Sci-Fi|Thriller']

,index,movieId,title,genres
5556,5556,26701,Patlabor: The Movie (Kidô keisatsu patorebâ: T...,Action|Animation|Crime|Drama|Film-Noir|Mystery...


In [ ]:
แล้วทำไม predict แต่ละครั้งได้หนังออกมาไม่เหมือนกัน?
movie_embedding มันไม่ได้ย้ายที่
user_embedding ก็ไม่ได้ย้ายที่

มันเป็นธรรมชาติของ neural network ที่จะ stochastic ค่ะ
คือแต่ละขั้นมันจะมีส่วนเล็กๆที่ทำให้การตัดสินใจแต่ละครั้งไม่เหมือนกัน
แต่ไม่ใช่ random นะ


In [ ]:
อะ พอเอา watched_movie_id_list มาจาก user input ละ
เดี๋ยวต้องไปเขียนเป็น function อีกมั้ยเนี่ย?

In [9]:
not_all_ratings = all_ratings[~all_ratings['movieId'].isin(watched_movie_id_list)]
not_all_ratings

,userId,movieId,title,rating
0,1,1,Toy Story (1995),4.0
1,1,3,Grumpier Old Men (1995),4.0
2,1,6,Heat (1995),4.0
3,1,47,Seven (a.k.a. Se7en) (1995),5.0
4,1,50,"Usual Suspects, The (1995)",5.0
...,...,...,...,...
100831,610,166534,Split (2017),4.0
100832,610,168248,John Wick: Chapter Two (2017),5.0
100833,610,168250,Get Out (2017),5.0
100834,610,168252,Logan (2017),5.0


In [10]:
not_all_ratings[80000:80020]

,userId,movieId,title,rating
80294,506,41571,Memoirs of a Geisha (2005),3.0
80295,506,45447,"Da Vinci Code, The (2006)",0.5
80296,506,45720,"Devil Wears Prada, The (2006)",3.0
80297,506,45880,Marie Antoinette (2006),3.0
80298,506,47629,The Queen (2006),4.0
80299,506,48997,Perfume: The Story of a Murderer (2006),3.5
80300,506,49286,"Holiday, The (2006)",1.0
80301,506,51834,Becoming Jane (2007),3.0
80302,506,52973,Knocked Up (2007),3.5
80303,506,52975,Hairspray (2007),3.0


In [14]:
not_all_ratings['movieId'] ไม่ซ้ำกันเลยซักกะบรรทัดหรือเปล่า ถึงแม้ว่าจะเป็นหนังเรื่องเดียวกัน
ไม่หนิ
สำหรับหนังเรื่อง the Davinci Code ทุกบรรทัดเป็น 45447

KeyError: "None of [Index(['Toy Story (1995)', 'Grumpier Old Men (1995)', 'Heat (1995)',\n       'Seven (a.k.a. Se7en) (1995)', 'Usual Suspects, The (1995)',\n       'From Dusk Till Dawn (1996)', 'Bottle Rocket (1996)',\n       'Braveheart (1995)', 'Rob Roy (1995)', 'Canadian Bacon (1995)',\n       ...\n       'The Magnificent Seven (2016)', 'Blair Witch (2016)', '31 (2016)',\n       'Arrival (2016)', 'Rogue One: A Star Wars Story (2016)', 'Split (2017)',\n       'John Wick: Chapter Two (2017)', 'Get Out (2017)', 'Logan (2017)',\n       'The Fate of the Furious (2017)'],\n      dtype='object', length=100474)] are in the [index]"

In [15]:
not_all_ratings.loc[not_all_ratings['movieId'] == 45447]

,userId,movieId,title,rating
3434,21,45447,"Da Vinci Code, The (2006)",3.5
7828,52,45447,"Da Vinci Code, The (2006)",3.5
8932,62,45447,"Da Vinci Code, The (2006)",3.5
9943,64,45447,"Da Vinci Code, The (2006)",2.5
16592,105,45447,"Da Vinci Code, The (2006)",4.0
17536,111,45447,"Da Vinci Code, The (2006)",4.0
25535,177,45447,"Da Vinci Code, The (2006)",3.5
30362,212,45447,"Da Vinci Code, The (2006)",3.5
31976,219,45447,"Da Vinci Code, The (2006)",2.5
32760,222,45447,"Da Vinci Code, The (2006)",3.0


In [10]:
user_movie_ratings = pd.pivot_table(all_ratings, values='rating', index='userId', columns='movieId')
user_movie_ratings

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
user_movie_id_ratings_matrix = user_movie_ratings.fillna(0)
user_movie_id_ratings_matrix

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
number_of_genres = 10
m = NMF(n_components=number_of_genres)
m.fit(user_movie_id_ratings_matrix)

NMF(n_components=10)